In [50]:
## requires: pytorch, transformer, flash-attn
from transformers import AutoModel, AutoTokenizer
from scipy.spatial.distance import cosine
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [51]:
query = "I want to hear a story about a rabbit."
model_name = "openbmb/MiniCPM-Embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, 
                                  trust_remote_code=True, 
                                  attn_implementation="flash_attention_2", 
                                  torch_dtype=torch.float16).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.32it/s]


In [52]:
df = pd.read_csv('stories/stories_cn_wpwx_ebd.csv')
df['embedding'] = df.embedding.apply(eval).apply(np.array, dtype="f")
df

,title,text,category,story_url,combined,embedding
0,梦幻童话《橙子国王与巧克力屋》,来源：中国儿童文学网 作者：小豆儿\n 有一片美丽的森林，那里鲜花明媚，鸟语花香，住着许...,童话故事,http://www.wpwx.cn/news/tonghua/22318144942K96...,title: 梦幻童话《橙子国王与巧克力屋》|category: 童话故事|text: 来源...,"[0.012741431, 0.0004346067, -0.00034689298, -0..."
1,羊妈妈和她的好心邻居们,来源：中国儿童文学网 作者：王绪化\n 羊妈妈快要生小羊羔了，这一下子不但乐坏了羊妈妈，...,童话故事,http://www.wpwx.cn/news/tonghua/211118135030JB...,title: 羊妈妈和她的好心邻居们|category: 童话故事|text: 来源：中国儿...,"[-0.0058315764, -0.0039620646, -0.0043118433, ..."
2,小鸟开花店,来源：中国儿童文学网 作者：陈彦旭\n江苏省盐城市大丰区城东实验小学文学社读书班 陈彦旭\...,童话故事,http://www.wpwx.cn/news/tonghua/211118122043D8...,title: 小鸟开花店|category: 童话故事|text: 来源：中国儿童文学网 ...,"[-0.00490396, -0.022032931, 0.011376677, -0.02..."
3,小猫和公鸡,来源：中国儿童文学网 作者：唐孖欣\n江苏省盐城市大丰区城东实验小学文学社读书班 唐孖欣\...,童话故事,http://www.wpwx.cn/news/tonghua/21111812191118...,title: 小猫和公鸡|category: 童话故事|text: 来源：中国儿童文学网 ...,"[0.0007365666, 0.044062275, 0.006877852, 0.020..."
4,小白兔和小青蛙,来源：中国儿童文学网 作者：吕金凇\n江苏省盐城市大丰区城东实验小学小海星文学社读书班 吕...,童话故事,http://www.wpwx.cn/news/tonghua/2111181217327C...,title: 小白兔和小青蛙|category: 童话故事|text: 来源：中国儿童文学网...,"[0.030266427, -0.0044559836, 0.023091758, 0.00..."
...,...,...,...,...,...,...
1113,女娲补天,来源：中国民间故事网 作者：佚名\n 有一天，大龙和精卫、小太极一起到远古时代去玩，居然...,神话故事,http://www.wpwx.cn/news/shenhua/07102010574F0H...,title: 女娲补天|category: 神话故事|text: 来源：中国民间故事网 作...,"[0.009518744, 0.02888613, 0.029543761, 0.02468..."
1114,白氏郎,来源：中国民间故事网 作者：佚名\n\n泰山周围有吕洞宾三戏白牡丹的传说，据说他们还生了个...,神话故事,http://www.wpwx.cn/news/shenhua/071020105620DJ...,title: 白氏郎|category: 神话故事|text: 来源：中国民间故事网 作者...,"[-0.003658157, 0.018223377, 0.006127568, -0.02..."
1115,鲤鱼跳龙门,来源：中国民间故事网 作者：佚名\n 庙峡，又名妙峡。两座巍峨雄奇的凤凰大山，拔水擎...,神话故事,http://www.wpwx.cn/news/shenhua/0741319383192K...,title: 鲤鱼跳龙门|category: 神话故事|text: 来源：中国民间故事网 ...,"[-0.0194253, 0.0165187, -0.0020781045, -0.0029..."
1116,盘古开天辟地,来源：中国民间故事网 作者：佚名\n在遥远的太古时代，宇宙好像一颗硕大无比的鸡蛋，里面漆黑...,神话故事,http://www.wpwx.cn/news/shenhua/07413193716DJ9...,title: 盘古开天辟地|category: 神话故事|text: 来源：中国民间故事网 ...,"[-0.002925636, 0.004189149, 0.022447756, 0.054..."


In [53]:
def weighted_mean_pooling(hidden, attention_mask):
    attention_mask_ = attention_mask * attention_mask.cumsum(dim=1)
    s = torch.sum(hidden * attention_mask_.unsqueeze(-1).float(), dim=1)
    d = attention_mask_.sum(dim=1, keepdim=True).float()
    reps = s / d
    return reps

@torch.no_grad()
def encode(input_texts):
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt', return_attention_mask=True).to("cuda")
    
    outputs = model(**batch_dict)
    attention_mask = batch_dict["attention_mask"]
    hidden = outputs.last_hidden_state

    reps = weighted_mean_pooling(hidden, attention_mask)   
    embeddings = F.normalize(reps, p=2, dim=1).detach().cpu().numpy()
    return embeddings

In [54]:
def find_kNN(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - cosine(x, y),
    top_n: int = 5
) -> tuple[list[str], list[float]]:
    query_embedding = encode([query])[0]
    df['similarity'] = df.embedding.apply(lambda x: relatedness_fn(query_embedding, x))

    idx = df.similarity.nlargest(top_n).index
    similarity = df.loc[idx, ['similarity']]
    content = df.loc[idx, ['combined']]
    
    return content.values, similarity.values

In [55]:
strings, relatednesses = find_kNN(query, df, top_n=5)
print('='*(len(query)+13))
print(f"|| QUERY: {query} ||")
print('='*(len(query)+13))
print("\nSearch result:")
print('-'*50)
for i, (string, relatedness) in enumerate(zip(strings, relatednesses), start=1):
    print(f"Result {i}:")
    print(f"Relatedness: {relatedness}")
    print(f"\nContent: {string}")
    print('-'*50)

|| QUERY: I want to hear a story about a rabbit. ||

Search result:
--------------------------------------------------
Result 1:
Relatedness: [0.46489504]

Content: ['title: 小白兔吃萝卜|category: 儿童故事|text: 来源：中国儿童文学网\u3000\u3000作者：人间怪才\n\u3000\u3000一连下了两场大雪，把天地变成了一样的颜色。白茫茫的雪海，美丽了整个大地。\u3000\u3000这天，可爱的小白兔对兔妈妈说：\u3000\u3000“妈妈，妈妈。我想吃又甜又脆的胡萝卜。”\u3000\u3000兔妈妈看着门前白茫茫铺天盖地的大雪说：\u3000\u3000“乖宝宝，现在外面下这么大的雪，都把路都掩盖住了，你让妈妈上哪给你找胡萝卜呢？再说吧！现在外面一定大黑狗正带着伙伴同他们的主人在满天雪地里找我们呢？我们如果出去被他们发现了，咱们就得去见阎王爷了。”\u3000\u3000小白兔一心想吃又甜又脆的胡萝卜，他才不理会兔妈妈所说的话呢？他便趁着兔妈妈一个不留神，快速的跑到外面。\u3000\u3000小白兔来到外面，看到白茫茫一片淹没自己的大雪，一下傻眼了。不知道自己该往哪里去寻找自己爱吃的胡萝卜，便坐在地上伤心起来。\u3000\u3000正在这个时候，他听到了秋天里他在玉米地里碰到的小黑狗的声音，由于，小黑狗爸爸妈妈不在身边，他们还交上了朋友，只要小黑狗一到玉米地，小白兔就出来找他玩耍。他们玩耍的可开心了。\u3000\u3000后来这件事情被兔妈妈知道了，她十分担心兔宝宝的安全，就向兔宝宝讲了外面的事情。可是小白兔没有别的朋友，也十分喜欢同小黑狗玩耍，于是，小白兔就把妈妈说的话告诉了小黑狗。小黑狗知道了，为了避免小白兔被自己的爸爸妈妈发现，就约了暗号，每次都离开自己的爸爸妈妈很远的地方同小白兔一起开开心心的玩耍，从来没有被爸爸妈妈发现过。\u3000\u3000小白兔听到小黑的声音，一下子来了精神，急忙跳了一下，便同小黑狗打起了暗号。过了不一时，小黑狗开心的跑过来，两个小伙伴见了十分开心。\u3000\u3000小白兔问小黑狗外面这么大的雪，他不待在家，出来干什么